# Import Library


In [ ]:
# Import library utama
!pip install wget
import wget
import zipfile
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=9cc96e257fb5b38ab8f59fef2550db7906e67a5a30a5e5c59e85afcd0fafdfb3
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


# Download dan Ekstraksi Dataset

In [ ]:
# URL file zip
url = "https://zenodo.org/records/7224690/files/FruQ-multi.zip?download=1"
zip_file_name = "FruQ-multi.zip"  # Nama file zip setelah diunduh
extract_to_folder = "datasetbuah"  # Folder tujuan untuk ekstraksi

# Download file zip
print("Downloading file...")
wget.download(url, zip_file_name)
print(f"\nFile downloaded: {zip_file_name}")

# Ekstrak file zip
print("Extracting files...")
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)
print(f"Files extracted to folder: {extract_to_folder}")

# Opsional: Hapus file zip setelah ekstraksi
if os.path.exists(zip_file_name):
    os.remove(zip_file_name)
    print(f"Removed zip file: {zip_file_name}")



File downloaded: FruQ-multi.zip
Extracting files...
Files extracted to folder: datasetbuah
Removed zip file: FruQ-multi.zip


# EDA

In [ ]:
# Path utama dataset
main_folder = "datasetbuah/FruQ-multi"

# List file di dalam folder
files = os.listdir(main_folder)
print(f"Files in '{main_folder}':")
print(files)

# Hitung jumlah gambar per kategori
fruit_data = {}

for fruit_folder in os.listdir(main_folder):
    fruit_path = os.path.join(main_folder, fruit_folder)
    if os.path.isdir(fruit_path):
        categories = {}

        for category_folder in os.listdir(fruit_path):
            category_path = os.path.join(fruit_path, category_folder)
            if os.path.isdir(category_path):
                num_images = len([f for f in os.listdir(category_path) if f.endswith(('.png', '.jpg', '.jpeg'))])
                categories[category_folder] = num_images

        fruit_data[fruit_folder] = categories

# Cetak hasil verifikasi data
print("Data per kategori:")
for fruit, categories in fruit_data.items():
    print(f"{fruit}: {categories}")


Files in 'datasetbuah/FruQ-multi':
['StrawberryQ', 'PearQ', 'PapayaQ', 'PepperQ', 'GrapeQ', 'tomatoQ', 'KakiQ', 'CucumberQ', 'WatermeloQ', 'BananaDB', 'PeachQ']
Data per kategori:
StrawberryQ: {'Mild': 119, 'Rotten': 97}
PearQ: {'Good': 504, 'Mild': 493, 'Rotten': 100}
PapayaQ: {'Good': 130, 'Mild': 250, 'Rotten': 413}
PepperQ: {'Good': 48, 'Mild': 24, 'Rotten': 660}
GrapeQ: {'Good': 227, 'Mild': 194, 'Rotten': 288}
tomatoQ: {'Good': 600, 'Mild': 440, 'Rotten': 950}
KakiQ: {'Good': 545, 'Mild': 226, 'Rotten': 340}
CucumberQ: {'Mild': 345, 'Fresh': 250, 'Rotten': 116}
WatermeloQ: {'Good': 51, 'Mild': 53, 'Rotten': 150}
BananaDB: {'Good': 179, 'mild': 96, 'Rotten': 337}
PeachQ: {'Good': 425, 'Mild': 136, 'Rotten': 584}


In [ ]:
# Dataset awal
dataset = {
    "WatermeloQ": {"Mild": 53, "Rotten": 150, "Good": 51},
    "StrawberryQ": {"Mild": 119, "Rotten": 97},
    "PepperQ": {"Mild": 24, "Rotten": 660, "Good": 48},
    "GrapeQ": {"Mild": 194, "Rotten": 288, "Good": 227},
    "BananaDB": {"mild": 96, "Rotten": 337, "Good": 179},
    "PeachQ": {"Mild": 136, "Rotten": 584, "Good": 425},
    "PearQ": {"Mild": 493, "Rotten": 100, "Good": 504},
    "tomatoQ": {"Mild": 440, "Rotten": 950, "Good": 600},
    "CucumberQ": {"Mild": 345, "Rotten": 116, "Fresh": 250},
    "PapayaQ": {"Mild": 250, "Rotten": 413, "Good": 130},
    "KakiQ": {"Mild": 226, "Rotten": 340, "Good": 545}
}

# Filter dataset valid
def is_valid_data(fruit_data):
    if not all(key in fruit_data for key in ["Good", "Mild", "Rotten"]):
        return False
    return min(fruit_data["Good"], fruit_data["Mild"], fruit_data["Rotten"]) > 20

filtered_dataset = {}
for fruit, data in dataset.items():
    corrected_data = {key.capitalize(): value for key, value in data.items()}
    if is_valid_data(corrected_data):
        filtered_dataset[fruit] = corrected_data

print("Dataset akhir yang siap digunakan:")
for fruit, data in filtered_dataset.items():
    print(f"{fruit}: {data}")


Dataset akhir yang siap digunakan:
WatermeloQ: {'Mild': 53, 'Rotten': 150, 'Good': 51}
PepperQ: {'Mild': 24, 'Rotten': 660, 'Good': 48}
GrapeQ: {'Mild': 194, 'Rotten': 288, 'Good': 227}
BananaDB: {'Mild': 96, 'Rotten': 337, 'Good': 179}
PeachQ: {'Mild': 136, 'Rotten': 584, 'Good': 425}
PearQ: {'Mild': 493, 'Rotten': 100, 'Good': 504}
tomatoQ: {'Mild': 440, 'Rotten': 950, 'Good': 600}
PapayaQ: {'Mild': 250, 'Rotten': 413, 'Good': 130}
KakiQ: {'Mild': 226, 'Rotten': 340, 'Good': 545}


# Prepocessing

In [ ]:
# Direktori dataset
train_dir = 'datasetbuah/FruQ-multi'
val_dir = 'datasetbuah/FruQ-multi'

# Augmentasi data untuk pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Generator data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical'
)


Found 9370 images belonging to 11 classes.
Found 9370 images belonging to 11 classes.


# Build Model

In [ ]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Bekukan model dasar
base_model.trainable = False

# Tambahkan lapisan tambahan
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


<ipython-input-11-16ce65daac26>:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Latih Model

In [ ]:
# Latih model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)


Epoch 1/5
586/586 ━━━━━━━━━━━━━━━━━━━━ 861s 1s/step - accuracy: 0.9332 - loss: 0.2534 - val_accuracy: 1.0000 - val_loss: 4.1610e-05
Epoch 2/5
586/586 ━━━━━━━━━━━━━━━━━━━━ 831s 1s/step - accuracy: 0.9949 - loss: 0.0143 - val_accuracy: 1.0000 - val_loss: 2.7353e-06
Epoch 3/5
586/586 ━━━━━━━━━━━━━━━━━━━━ 0s 832ms/step - accuracy: 0.9973 - loss: 0.0072